In [1]:
import scipy.io as sio
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import math
import time
import torch
from torch import nn
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import math
import time
from sklearn.preprocessing import MaxAbsScaler

import visdom
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm
from model.multi_scale_ori import *
from imblearn.over_sampling import SMOTE
from PIL import Image
from utils import *
from preprocessing import pre
from config import cfg
import datetime

In [2]:
class FocalLoss(torch.nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss

In [3]:
def main():
    vis = visdom.Visdom(env="resnet1D_focal_loss_2", port=8097)
    
    data = pre(train_data_path=cfg.train_data_path, 
           test_data_path=cfg.test_data_path,
           y_train_path=cfg.train_label, 
           y_val_path=cfg.test_label,
           batch_size=cfg.batch_size, 
           aspect_ratio=cfg.aspect_ratio)
    
    #clear gpu_cache
    torch.cuda.empty_cache() 

    #set parameter 
    #class　
    n_class = cfg.n_class

    # Number of epochs to train for
    num_epochs = cfg.num_epochs

    # Flag for feature extracting. When False, we finetune the whole model,
    # when True we only update the reshaped layer params
    feature_extract = False

    #use gpu
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    #learning rate 
    lr=cfg.learn_rate
    msresnet = MSResNet(input_channel=cfg.channel, layers=cfg.resnet_layer, num_classes=n_class)

    # use GPU
    msresnet.to(device)

    #weight decay
    weight_decay=cfg.weight_decay
    
    params_to_update = msresnet.parameters()

    #Observe that all parameters are being optimized
    optimizer = torch.optim.Adam(msresnet.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 100, 150, 200, 250, 300], gamma=0.1)
    criterion = FocalLoss(logits=True)
     
    #train_model
    currunt_turn = 0

    for epoch in range(num_epochs):
    
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 100)
     
        for epoch_i, (inputs_nonacc, labels_nonacc) in enumerate(data.train_nonacc_dataloader):
        
            msresnet.train() 
            currunt_turn += 1
        
            #sampling_acc, labels_acc
            inputs_acc, labels_acc = next(iter(data.train_acc_dataloader))
        
            #concat_sampling 
            inputs = torch.cat([inputs_nonacc, inputs_acc], axis=0) 
            labels = torch.cat([labels_nonacc, labels_acc], axis=0) 
        
            #shuffle inputs and labels 
            shuffled_index = torch.randperm(cfg.batch_size + cfg.aspect_ratio * cfg.batch_size)
            inputs = inputs[shuffled_index]
            labels = labels[shuffled_index]
        
            #use GPU
            inputs, labels = Variable(inputs.type(torch.FloatTensor).cuda()), Variable(labels.float().cuda())
        
            #init optimizer
            optimizer.zero_grad()
            
            #print(inputs.shape)
            #zeo_padd =  torch.zeros(200, 735, 3).cuda()
            #inputs = torch.cat([zeo_padd, inputs], dim=1)
            #print(inputs.shape)
            
            #print(inputs.shape)
            p1d = (0,0,0,735) # pad last dim by 1 on each side
            inputs = F.pad(inputs, p1d, "constant", 0) 
            #print(inputs.shape)
            
            #calc output
            #outputs, _ = msresnet(inputs)
            outputs, _ = msresnet(inputs.permute(0, 2, 1))
            loss = criterion(outputs, labels)
        
            #calc_pred_label
            outputs = torch.sigmoid(outputs)
            tag_one= torch.ones_like(outputs)
            tag_zero= torch.zeros_like(outputs)
            preds = torch.where(outputs >= 0.5, tag_one, tag_zero)
        
            preds = preds.data.cpu().numpy()
            labels = labels.data.cpu().numpy()
            
            #calc f1 scores
            precision = precision_score(labels, preds)
            recall = recall_score(labels, preds)
            f1 = f1_score(labels, preds)
        
            visulization(vis, ptype= "line",X=currunt_turn, Y=f1, win_name="f1")       
            visulization(vis, ptype= "line",X=currunt_turn, Y=loss.item(), win_name="loss")
        
            #backward 
            loss.backward()
            optimizer.step() 
        
            if epoch_i % 200 == 0: 
                dt_now = datetime.datetime.now()
                #torch.save(msresnet.state_dict(), cfg.training_model_path + "model_save/model_{}_{}.pth".format(str(epoch) + "_" + str(epoch_i), str(dt_now.month) + "_" + str(dt_now.day)))
                outputs_val_all, preds_val_all, labels_val_all, loss_val_all = cal_train_data(msresnet=msresnet, dataloader=data.test_dataloader, criterion=criterion)
                outputs_train_all, preds_train_all, labels_train_all, loss_train_all = cal_train_data(msresnet=msresnet, dataloader=data.train_dataloader, criterion=criterion)
            
                loss_mean_test, f1_val = eval_metric(loss_val_all, labels_val_all, preds_val_all, epoch, epoch_i, phaze="test")
                loss_mean_train, f1_train = eval_metric(loss_train_all, labels_train_all, preds_train_all, epoch, epoch_i, phaze="train")
            
                visulization(vis, ptype= "line",X=currunt_turn, Y=loss_mean_test, win_name="loss_test")
                visulization(vis, ptype= "line",X=currunt_turn, Y=f1_val, win_name="f1_test")
                visulization(vis, ptype= "line",X=currunt_turn, Y=loss_mean_train, win_name="loss_train")
                visulization(vis, ptype= "line",X=currunt_turn, Y=f1_train, win_name="f1_train")
        
                plot_confusion_matrix(outputs_val_all, labels_val_all, fig_title=str(epoch) + "_" + str(epoch_i))

In [4]:
main()

Setting up a new session...


train_data_shape: (67682, 750, 3)
test_data_shape: (16881, 750, 3)
y_train_shape: 3305
y_val_shape: 827
Epoch 0/1
----------------------------------------------------------------------------------------------------


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


KeyboardInterrupt: 

# 結果検証

In [ ]:
import pandas as pd
pd.to_pickle(testdata_compare_scsk, "./model_result/demodata.pkl")
pd.to_pickle(y_val_compare_scsk, "./model_result/demodata_y.pkl")
pd.to_pickle(selected_id, "./model_result/noacc_id.pkl")
pd.to_pickle(test_acc_index, "./model_result/acc_id.pkl")

In [ ]:
def find_index_error(thr): 
    return np.where((np.array(labels_val_all) != np.array(preds_val_all)) & (np.array(outputs_val_all) <= thr))

def find_index_error_bt_prob(thr, thr2): 
    return np.where((np.array(labels_val_all) != np.array(preds_val_all)) &
                    (np.array(outputs_val_all) >= thr) &
                    (np.array(outputs_val_all) <= thr2))

index_error_001, _ = find_index_error(0.01)
index_error_090, _ = find_index_error_bt_prob(0.9, 1)
index_error_0506, _ = find_index_error_bt_prob(0.5, 0.6)
index_error_0405, _ = find_index_error_bt_prob(0.4, 0.5)

pd.to_pickle(index_error_001, "./model_result/index_test_error_001.pkl")
pd.to_pickle(index_error_090, "./model_result/index_test_error_090.pkl")
pd.to_pickle(index_error_0506, "./model_result/index_test_error_0506.pkl")
pd.to_pickle(index_error_0405, "./model_result/index_test_error_0405.pkl")

In [ ]:
#間違ったデータを主力
#index_test_error, _= np.where(np.array(labels_val_all) != np.array(preds_val_all))
#index_train_error, _= np.where(np.array(labels_train_all) != np.array(preds_train_all))

#間違ったデータのインデックスを出力
#pd.to_pickle(index_error, "./model_result/index_train_error.pkl")
#pd.to_pickle(index_error, "./model_result/index_test_error.pkl")

#予測した確率を出力
pd.to_pickle(outputs_train_all, "./model_result/output_train_1.pkl")

#予測した確率を出力
pd.to_pickle(outputs_val_all, "./model_result/output_val_1.pkl")

In [ ]:
inputs

In [ ]:
zeo_padd =  torch.zeros(200, 735, 3).cuda()

In [ ]:
torch.cat([zeo_padd, t4d.cuda()], dim=1)[0,: ,0]

In [ ]:
t4d = torch.ones(200, 15, 3)
p1d = (0,0,0,735) # pad last dim by 1 on each side
out = F.pad(t4d, p1d, "constant", 0)  # effectively zero padding
print(out.size())